In [9]:
import pandas as pd
import numpy as np

# Read the csv file
df = pd.read_csv('dataset.csv')

# Map the labels to binary values
df['label'] = df['label'].map({'NORMAL': 0, 'PNEUMONIA': 1})

# Shuffle the dataframe
df = df.sample(frac=1, random_state=42)

# Select the first 5000 rows as the training set
train_df = df.head(5000)

# Select the remaining rows as the validation set
val_df = df.tail(df.shape[0] - 5000)

x_train = train_df.iloc[:,1:].values
y_train = train_df.iloc[:,0].values
x_val = val_df.iloc[:,1:].values
y_val = val_df.iloc[:,0].values

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Preprocess the data
x_train = (x_train - (np.min(x_train))) / (np.max(x_train) - (np.min(x_train)))
x_val = (x_val - (np.min(x_val))) / (np.max(x_val) - (np.min(x_val)))
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train = np.repeat(x_train, 3, axis=-1)
x_val = np.repeat(x_val, 3, axis=-1)
x_train = x_train.reshape(-1, 100, 100, 3)
x_val = x_val.reshape(-1, 100, 100, 3)

In [16]:
for i in [x_train,y_train,x_val,y_val]:
    print(i.shape)

(5000, 100, 100, 3)
(5000,)
(232, 100, 100, 3)
(232,)


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [14]:
# Load the DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add a new classifier layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on your dataset
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=10,
                    validation_data=(x_val, y_val))

29097984/29084464 [==============================] - 2s 0us/step
Epoch 1/10


C:\Users\LEOFEN~1\AppData\Local\Temp/ipykernel_211064/1060430081.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),


156/156 [==============================] - 50s 295ms/step - loss: 0.2894 - accuracy: 0.9044 - val_loss: 0.1266 - val_accuracy: 0.9353
Epoch 2/10
156/156 [==============================] - 45s 287ms/step - loss: 0.1577 - accuracy: 0.9416 - val_loss: 0.1236 - val_accuracy: 0.9526
Epoch 3/10
156/156 [==============================] - 45s 289ms/step - loss: 0.1428 - accuracy: 0.9470 - val_loss: 0.1289 - val_accuracy: 0.9397
Epoch 4/10
156/156 [==============================] - 45s 285ms/step - loss: 0.1427 - accuracy: 0.9478 - val_loss: 0.1558 - val_accuracy: 0.9267
Epoch 5/10
156/156 [==============================] - 43s 278ms/step - loss: 0.1267 - accuracy: 0.9518 - val_loss: 0.1055 - val_accuracy: 0.9526
Epoch 6/10
156/156 [==============================] - 45s 287ms/step - loss: 0.1241 - accuracy: 0.9554 - val_loss: 0.2123 - val_accuracy: 0.9224
Epoch 7/10
156/156 [==============================] - 45s 289ms/step - loss: 0.1276 - accuracy: 0.9506 - val_loss: 0.1961 - val_accuracy: 0.9

In [15]:
# Evaluate the model on the validation set
scores = model.evaluate(x_val, y_val, verbose=0)

# Print the accuracy
print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 93.97%
